In [1]:
import pandas as pd
import numpy as np
import bqplot
import traitlets
import ipywidgets

In [2]:
buildings = pd.read_csv("building_inventory.csv", 
                        na_values={"Year Acquired": 0,
                                   "Year Constructed": 0,
                                   "Square Footage": 0})

buildings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8862 entries, 0 to 8861
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Agency Name              8862 non-null   object 
 1   Location Name            8862 non-null   object 
 2   Address                  8811 non-null   object 
 3   City                     8862 non-null   object 
 4   Zip code                 8862 non-null   int64  
 5   County                   8837 non-null   object 
 6   Congress Dist            8862 non-null   int64  
 7   Congressional Full Name  8699 non-null   object 
 8   Rep Dist                 8862 non-null   int64  
 9   Rep Full Name            8839 non-null   object 
 10  Senate Dist              8862 non-null   int64  
 11  Senator Full Name        8839 non-null   object 
 12  Bldg Status              8862 non-null   object 
 13  Year Acquired            8597 non-null   float64
 14  Year Constructed        

In [3]:
# Show values in Year Acquired column
buildings.head()[['Year Acquired']]

,Year Acquired
0,1975.0
1,2004.0
2,2004.0
3,2004.0
4,2004.0


In [4]:
# Convert without format
buildings_cp = buildings.copy()
buildings_cp['Year Acquired'] = pd.to_datetime(buildings_cp['Year Acquired'])
buildings_cp.head()[['Year Acquired']]

,Year Acquired
0,1970-01-01 00:00:00.000001975
1,1970-01-01 00:00:00.000002004
2,1970-01-01 00:00:00.000002004
3,1970-01-01 00:00:00.000002004
4,1970-01-01 00:00:00.000002004


In [6]:
buildings_cp['Year Acquired'].dt.year.tolist()[:3]

[1970.0, 1970.0, 1970.0]

In [7]:
buildings_cp['Year Acquired'].dt.microsecond.tolist()[:3]

[1.0, 2.0, 2.0]

In [5]:
# Convert with format
buildings['Year Acquired'] = pd.to_datetime(buildings['Year Acquired'], 
                                            format='%Y')
print(buildings['Year Acquired'].dtype)
buildings.head()[['Year Acquired']]

datetime64[ns]


,Year Acquired
0,1975-01-01
1,2004-01-01
2,2004-01-01
3,2004-01-01
4,2004-01-01


In [8]:
# Extract the year from the date object, Note the datatype
buildings['Year Acquired'] = buildings['Year Acquired'].dt.year
print(buildings['Year Acquired'].dtype)
buildings.head()[['Year Acquired']]

float64


,Year Acquired
0,1975.0
1,2004.0
2,2004.0
3,2004.0
4,2004.0


# UFO dataset

In [9]:
#!wget https://github.com/planetsig/ufo-reports/raw/master/csv-data/ufo-scrubbed-geocoded-time-standardized.csv

In [10]:
ufo = pd.read_csv('ufo-scrubbed-geocoded-time-standardized.csv', 
                  names=['date_sighted', 'city', 'state', 'country',
                         'shape', 'duration', 
                         'duration_txt', 'note', 'date_reported', 
                         'latitude', 'longitude'],
                  parse_dates=['date_sighted', 'date_reported'])
ufo = ufo.reset_index().rename(columns={'index':'ufo_id'})
print(ufo.shape)

ufo = ufo.loc[~ufo.index.isin([27822, 35692, 58591, 43782])] # Ignore dirty data for now

df = ufo.sample(n=1000, random_state=5).reset_index(drop=True)
df['date_sighted'] = df['date_sighted'].str.replace('24:00', '00:00') 
df['duration'] = df['duration'].astype(float)
df['latitude'] = df['latitude'].astype(float)
df['date_sighted'] = pd.to_datetime(df['date_sighted'])

print(df.info())
df.head()

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(80332, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ufo_id         1000 non-null   int64         
 1   date_sighted   1000 non-null   datetime64[ns]
 2   city           1000 non-null   object        
 3   state          922 non-null    object        
 4   country        866 non-null    object        
 5   shape          974 non-null    object        
 6   duration       1000 non-null   float64       
 7   duration_txt   1000 non-null   object        
 8   note           1000 non-null   object        
 9   date_reported  1000 non-null   datetime64[ns]
 10  latitude       1000 non-null   float64       
 11  longitude      1000 non-null   float64       
dtypes: datetime64[ns](2), float64(3), int64(1), object(6)
memory usage: 93.9+ KB
None


,ufo_id,date_sighted,city,state,country,shape,duration,duration_txt,note,date_reported,latitude,longitude
0,60887,2010-07-30 23:05:00,uk/scotland,NaN,NaN,light,180.0,3 mins,two bright lights travelling at speed across a...,2010-08-24,56.490671,-4.202646
1,5167,2011-10-28 16:39:00,scottsdale,az,us,unknown,300.0,5min or less,Three to Five comet-like objects seen descendi...,2011-12-12,33.509167,-111.898333
2,45720,2007-05-05 01:00:00,denton,ne,us,other,420.0,5-7 minutes,Object seen outside of Lincoln &#44NE,2007-06-12,40.737778,-96.844167
3,40365,2006-04-05 22:25:00,oklahoma city,ok,us,circle,600.0,10 minutes,Two hovering orange circles drop third orange ...,2006-05-15,35.467500,-97.516111
4,28803,2003-02-27 00:00:00,patoka,in,us,unknown,300.0,5 minutes off and on,on a morning jog me and my friend saw lights i...,2003-03-21,38.406944,-87.585556


# Time-series data

In [11]:
# prep data: Number of UFOs sighted in each month
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

line_data = df.groupby(pd.Grouper(key='date_sighted', 
                                  freq='M'))[['ufo_id']].count()
line_data

,ufo_id
date_sighted,
1955-06-30,1
1955-07-31,0
1955-08-31,0
1955-09-30,0
1955-10-31,0
...,...
2014-01-31,8
2014-02-28,2
2014-03-31,8


In [14]:
# Check if grouped result is correct
df.loc[df['date_sighted'].dt.year == 1955]

,ufo_id,date_sighted,city,state,country,shape,duration,duration_txt,note,date_reported,latitude,longitude
330,48587,1955-06-15 04:30:00,missouri (northwest part),mo,NaN,light,300.0,5 min,I will never forget this this experience. The ...,2003-03-21,37.964253,-91.831833


In [15]:
line_data.loc[line_data.index.year == 1955]

,ufo_id
date_sighted,
1955-06-30,1
1955-07-31,0
1955-08-31,0
1955-09-30,0
1955-10-31,0
1955-11-30,0
1955-12-31,0


## Parameter: X lim

In [17]:
# range slider for years
my_slider = ipywidgets.SelectionRangeSlider(options=line_data.index.year,
                                            description='Year Range', 
                                            layout={'width':"600px"})
my_slider

SelectionRangeSlider(description='Year Range', index=(0, 0), layout=Layout(width='600px'), options=(1955, 1955…

In [19]:
my_slider.value

(1989, 1998)

In [23]:
# A line plot + displayed the selected year window

# Scale
x_sc = bqplot.DateScale()
y_sc = bqplot.LinearScale()

# Axis
x_ax = bqplot.Axis(scale=x_sc, label='Date Sighted')
y_ax = bqplot.Axis(scale=y_sc, label='UFO counts', 
                   orientation='vertical')

# Mark
lines = bqplot.Lines(x=line_data.index, 
                     y=line_data['ufo_id'], 
                     scales={'x':x_sc, 'y':y_sc})

# Interaction
my_slider = ipywidgets.SelectionRangeSlider(options=line_data.index.year,
                                            description='Year Range', 
                                            layout={'width':"600px"})

def slider_func(change):
    yr_range = my_slider.value
    #print(yr_range)
    
    yr_start, yr_end = yr_range
    
    filter_1 = line_data.index.year >= yr_start
    filter_2 = line_data.index.year <= yr_end
    
    line_data_selected = line_data.loc[(filter_1)&(filter_2)]
    
    lines.x = line_data_selected.index
    lines.y = line_data_selected['ufo_id']

my_slider.observe(slider_func, 'value')

# Fig
line_fig = bqplot.Figure(marks=[lines], axes=[x_ax, y_ax])
line_fig.layout.height='200px'
line_fig_slider = ipywidgets.VBox([my_slider, line_fig])
line_fig_slider

# Heatmap and click selected

In [24]:
# Prep data: UFOs reported in different years and countries
heatmap_data = df.groupby([df['date_reported'].dt.year, 'country'])[['ufo_id']].count()
heatmap_data = heatmap_data.reset_index()
heatmap_data = heatmap_data.pivot(index='date_reported', 
                                  columns='country', 
                                  values='ufo_id')
heatmap_data

country,au,ca,de,gb,us
date_reported,,,,,
1998,NaN,NaN,NaN,NaN,9.0
1999,NaN,NaN,NaN,NaN,50.0
2000,1.0,3.0,NaN,NaN,25.0
2001,NaN,2.0,NaN,2.0,31.0
2002,NaN,5.0,NaN,5.0,39.0
2003,1.0,2.0,NaN,3.0,47.0
2004,3.0,3.0,NaN,NaN,47.0
2005,NaN,NaN,NaN,4.0,55.0
2006,NaN,2.0,1.0,NaN,30.0


In [25]:
# Heatmap without interactivity
# Scale
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

clr = np.log10(heatmap_data)
c_sc = bqplot.ColorScale(scheme='BuPu', min=np.nanmin(clr) , max=np.nanmax(clr))

# Axis
x_ax = bqplot.Axis(scale=x_sc, label='Country')
y_ax = bqplot.Axis(scale=y_sc, label='Year Reported', orientation='vertical')
c_ax = bqplot.ColorAxis(scale=c_sc, side='right')

# Marks
heatmap = bqplot.GridHeatMap(column=heatmap_data.columns,
                             row=heatmap_data.index,
                             scales={'column':x_sc, 'row':y_sc, 'color':c_sc}, 
                             color=clr)

# Fig
heatmap_fig = bqplot.Figure(marks=[heatmap], axes=[x_ax, y_ax, c_ax])
heatmap_fig

Figure(axes=[Axis(label='Country', scale=OrdinalScale()), Axis(label='Year Reported', orientation='vertical', …

In [32]:
# Heatmap with click and select that shows the UFO counts in each month

# Left plot: heatmap
# Scale
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

clr = np.log10(heatmap_data)
c_sc = bqplot.ColorScale(scheme='BuPu', min=np.nanmin(clr) , max=np.nanmax(clr))

# Axis
x_ax = bqplot.Axis(scale=x_sc, label='Country')
y_ax = bqplot.Axis(scale=y_sc, label='Year Reported', orientation='vertical')
c_ax = bqplot.ColorAxis(scale=c_sc, side='right')

# Marks
heatmap = bqplot.GridHeatMap(column=heatmap_data.columns,
                             row=heatmap_data.index,
                             scales={'column':x_sc, 'row':y_sc, 'color':c_sc}, 
                             color=clr, 
                             interactions={'click':'select'}, 
                             selected_style={'fill':'green'})


# Right plot: line plot

# Scale 
x_sc_r = bqplot.DateScale()
y_sc_r = bqplot.LinearScale()

# Axis
x_ax_r = bqplot.Axis(scale=x_sc_r, label='Month')
y_ax_r = bqplot.Axis(scale=y_sc_r, label='UFO counts', 
                     orientation='vertical')

# Mark
lines = bqplot.Lines(scales={'x':x_sc_r, 'y':y_sc_r})


# Interaction:

def observe_func(change):
    selected_cell = heatmap.selected
    print(selected_cell)
    
    selected_row, selected_col = selected_cell[0]
    
    years = heatmap_data.index.tolist()
    countries = heatmap_data.columns.tolist()
    
    selected_yr = years[selected_row]
    selected_country = countries[selected_col]
    
    print(selected_yr, selected_country)
    
    filter_1 = df['date_reported'].dt.year == selected_yr
    filter_2 = df['country'] == selected_country
    
    df_selected = df.loc[(filter_1)&(filter_2)]
    
    df_selected_g = df_selected.groupby(pd.Grouper(key='date_reported', 
                                                   freq='M'))[['ufo_id']].count()
    
    lines.x = df_selected_g.index
    lines.y = df_selected_g['ufo_id']

heatmap.observe(observe_func, 'selected')


# Fig
heatmap_fig = bqplot.Figure(marks=[heatmap], axes=[x_ax, y_ax, c_ax])
line_fig = bqplot.Figure(marks=[lines], axes=[x_ax_r, y_ax_r], 
                         title='UFO counts by month in the selected year and country')

heatmap_fig.layout.width = '500px'
line_fig.layout.width = '500px'

# Dashboard
my_dashboard = ipywidgets.HBox([heatmap_fig, line_fig])
my_dashboard

[[10  4]]
2008 us
[[16  4]]
2014 us
[[9 4]]
2007 us
[[5 4]]
2003 us


# Maps

In [ ]:
# US map data

In [ ]:
# Data of one state


In [ ]:
# US Map with tooltip

In [ ]:
# World Map with tooltip

# US Map with customized colors

In [ ]:
!wget https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv

In [ ]:
# Values in the us-state-ansi-fips.csv

In [ ]:
# rename column names and clean values

In [ ]:
# FIPS and state names in the US State map data

## Coropleth map

In [ ]:
# Color map by number of UFOs sighted in states

In [ ]:
# Color: number of UFOs sighted in states

## A Dashboard showing duration in seconds by years in the selected state

In [ ]:
# Click select + sum of duration in seconds (all the selected states are combined)


In [ ]:
# Click select + sum of duration in seconds (the selected states presented as seperated lines)



## A Dashboard with three plots

In [ ]:
# Click select + sum of duration in seconds (the selected states are in seperated lines) + stacked bars by shapes